In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras
import os

In [3]:
from keras.models import Sequential
from keras.layers.core import Flatten,Dense,Dropout
from keras.layers.convolutional import Convolution2D,MaxPooling2D,ZeroPadding2D
from keras.optimizers import SGD
import cv2

In [4]:
df = pd.read_csv('fer2013.csv')

In [5]:
df.shape

(35887, 3)

In [6]:
df.head(-5)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35877,6,139 143 145 154 159 168 176 181 190 191 195 19...,PrivateTest
35878,3,0 39 81 80 104 97 51 64 68 46 41 67 53 68 70 5...,PrivateTest
35879,2,0 0 6 16 19 31 47 18 26 19 17 8 15 3 4 2 14 20...,PrivateTest
35880,2,164 172 175 171 172 173 178 181 188 192 197 20...,PrivateTest


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [8]:
df['Usage'].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64

In [7]:
X_train,y_train,X_test,y_test = [],[],[],[]

In [8]:
for index,row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val,'float32'))
            y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val,'float32'))
            y_test.append(row['emotion'])
    except:
        print("error at index :{0} and row:{1}".format(index,row))

In [9]:
print(len(X_train))


28709


In [12]:
X_test[:5]

[array([254., 254., 254., ...,  42., 129., 180.], dtype=float32),
 array([156., 184., 198., ..., 172., 167., 161.], dtype=float32),
 array([ 69., 118.,  61., ...,  88.,  87.,  90.], dtype=float32),
 array([205., 203., 236., ...,  43.,  82.,  86.], dtype=float32),
 array([87., 79., 74., ..., 41., 34., 32.], dtype=float32)]

In [13]:
y_test[0]

0

In [10]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 60
width, height = 48,48

In [11]:
X_train = np.array(X_train,'float32')
X_test = np.array(X_test,'float32')
y_train = np.array(y_train,'float32')
y_test = np.array(y_test,'float32')

In [12]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(28709, 2304)
(28709,)
(3589, 2304)
(3589,)


In [17]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train,num_classes=num_labels)
y_test = to_categorical(y_test,num_classes=num_labels)

In [18]:
print(y_train[0])

[1. 0. 0. 0. 0. 0. 0.]


In [19]:
print(y_train.shape)
print(y_test.shape)

(28709, 7)
(3589, 7)


In [20]:
#normalize dataset
X_train -= np.mean(X_train,axis=0)
X_train /= np.std(X_train,axis=0)

X_test -= np.mean(X_test,axis=0)
X_test /= np.std(X_test,axis=0)

X_train = X_train.reshape(X_train.shape[0],48,48,1)
X_test = X_test.reshape(X_test.shape[0],48,48,1)



In [21]:
print(X_train.shape)
print(X_test.shape)

(28709, 48, 48, 1)
(3589, 48, 48, 1)


In [22]:
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPooling2D,Activation
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import layers
from tensorflow import keras

In [23]:
#build the model
model = Sequential()

#1st layer
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape = X_train.shape[1:]))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

#2nd layer
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

#3rd layer
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())

#fully connected layers
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 9, 9, 64)          0

In [24]:
#compile model
model.compile(loss = categorical_crossentropy, 
              optimizer = 'sgd',
              metrics=['accuracy'])

In [26]:
print(keras.__version__)

2.4.0


In [27]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(28709, 48, 48, 1)
(3589, 48, 48, 1)
(28709, 7)
(3589, 7)


In [28]:
model.fit(X_train , y_train,
          batch_size = batch_size,
          epochs = epochs,
          verbose = 1,
          validation_data = (X_test , y_test),
          shuffle = True )

Epoch 1/60
449/449 [==============================] - 145s 324ms/step - loss: 1.8252 - accuracy: 0.2485 - val_loss: 1.8217 - val_accuracy: 0.2497
Epoch 2/60
449/449 [==============================] - 145s 323ms/step - loss: 1.7923 - accuracy: 0.2547 - val_loss: 1.7996 - val_accuracy: 0.2508
Epoch 3/60
449/449 [==============================] - 146s 326ms/step - loss: 1.7793 - accuracy: 0.2584 - val_loss: 1.7939 - val_accuracy: 0.2510
Epoch 4/60
449/449 [==============================] - 145s 323ms/step - loss: 1.7694 - accuracy: 0.2630 - val_loss: 1.7756 - val_accuracy: 0.2566
Epoch 5/60
449/449 [==============================] - 144s 322ms/step - loss: 1.7542 - accuracy: 0.2793 - val_loss: 1.7583 - val_accuracy: 0.2859
Epoch 6/60
449/449 [==============================] - 146s 325ms/step - loss: 1.7287 - accuracy: 0.2999 - val_loss: 1.7144 - val_accuracy: 0.3029
Epoch 7/60
449/449 [==============================] - 149s 332ms/step - loss: 1.6944 - accuracy: 0.3203 - val_loss: 1.6764 -

Epoch 57/60
449/449 [==============================] - 142s 316ms/step - loss: 1.0779 - accuracy: 0.5909 - val_loss: 1.1492 - val_accuracy: 0.5628
Epoch 58/60
449/449 [==============================] - 192s 427ms/step - loss: 1.0706 - accuracy: 0.5959 - val_loss: 1.1589 - val_accuracy: 0.5634
Epoch 59/60
449/449 [==============================] - 232s 517ms/step - loss: 1.0617 - accuracy: 0.5989 - val_loss: 1.1550 - val_accuracy: 0.5651
Epoch 60/60
449/449 [==============================] - 232s 516ms/step - loss: 1.0517 - accuracy: 0.6016 - val_loss: 1.1630 - val_accuracy: 0.5620


In [25]:
#save the model
fer_json = model.to_json()
with open("fer.json","w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

In [90]:
from keras.models import model_from_json
from keras.preprocessing import image

model = model_from_json(open("fer.json","r").read())
model.load_weights("fer.h5")
face_haar_cascade = cv2.CascadeClassifier("haar-cascade-files-master/haarcascade_frontalface_default.xml")

In [91]:
import cv2
test_image = cv2.imread("test2.jpg")
cv2.imshow('image',test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [92]:
gray_image = cv2.cvtColor(test_image,cv2.COLOR_BGR2GRAY)
cv2.imshow('grayscale_image',gray_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [93]:
face_haar_cascade = cv2.CascadeClassifier("haar-cascade-files-master/haarcascade_frontalface_default.xml")
from keras.preprocessing.image import img_to_array
cv2.imshow('image',test_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [94]:
faces = face_haar_cascade.detectMultiScale(gray_image,1.1,4)
for(x,y,w,h) in faces:
    cv2.rectangle(test_image,(x,y),(x+w,y+h),(255,0,0))
    roi_gray = gray_image[y:y+w,x:x+h]
    roi_gray = cv2.resize(roi_gray,(48,48))
    image_pixels = img_to_array(roi_gray)
    image_pixels = np.expand_dims(image_pixels,axis = 0)
    image_pixels /= 255
    predictions = model.predict(image_pixels)
    max_index = np.argmax(predictions[0])
    emotion_detection = ('angry','disgust','fear','happy','sad','surprise','neutral')
    emotion_prediction = emotion_detection[max_index]
    print(emotion_prediction)
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50,50)
    fontScale = 1
    color = (255,0,0)
    thickness = 2
    image = cv2.putText(test_image,emotion_prediction,org,font,fontScale,color,thickness,cv2.LINE_AA) 
    cv2.imshow('output',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    
    

disgust
